# FP8 Quantization-Aware Training (QAT) with Unsloth

This notebook demonstrates how to perform FP8 Quantization-Aware Training using the Unsloth library for efficient LLM fine-tuning.

## Install Dependencies

Install PyTorch with CUDA support and all required libraries including Unsloth, transformers, TRL (for training), and torchao/fbgemm for FP8 quantization support.

In [2]:
!pip install torch==2.9.0 torchvision==0.24.0 torchaudio==2.9.0 --index-url https://download.pytorch.org/whl/cu130

import re

import torch; v = re.match(r"[0-9]{1,}\.[0-9]{1,}", str(torch.__version__)).group(0)
xformers = "xformers==" + ("0.0.33.post1" if v=="2.9" else "0.0.32.post2" if v=="2.8" else "0.0.29.post3")
!pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf "datasets==4.3.0" "huggingface_hub>=0.34.0" hf_transfer
!pip install --no-deps unsloth
!pip install torchao==0.14.0 fbgemm-gpu-genai==1.4.2
!pip install transformers==4.55.4
!pip install --no-deps trl==0.22.2

Looking in indexes: https://download.pytorch.org/whl/cu130
  Using cached https://download.pytorch.org/whl/cu130/torch-2.9.0%2Bcu130-cp312-cp312-manylinux_2_28_aarch64.whl.metadata (30 kB)
  Using cached https://download.pytorch.org/whl/cu130/torchvision-0.24.0-cp312-cp312-manylinux_2_28_aarch64.whl.metadata (5.9 kB)
  Using cached https://download.pytorch.org/whl/cu130/torchaudio-2.9.0-cp312-cp312-manylinux_2_28_aarch64.whl.metadata (6.9 kB)
  Using cached filelock-3.20.0-py3-none-any.whl.metadata (2.1 kB)
  Using cached https://download.pytorch.org/whl/typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached https://download.pytorch.org/whl/setuptools-70.2.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.6.1-py3-none-any.whl.metadata (6.8 kB)
  Using cached https://download.pytorch.org/whl/jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2025.12.0-py3-none-any.whl.metada

## Load the Pre-trained Model

Load the Llama-3.1-8B-Instruct model using Unsloth's `FastLanguageModel` for efficient inference and fine-tuning. The model is loaded in bfloat16 precision with a maximum sequence length of 2048 tokens.

In [3]:
from unsloth import FastLanguageModel
MODEL_ID="meta-llama/Llama-3.1-8B-Instruct"
max_seq_length = 2048
dtype = torch.bfloat16
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_ID,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_16bit=True,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/root/src/github.com/elizabetht/language-modeling-from-scratch/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/root/src/github.com/elizabetht/language-modeling-from-scratch/.venv/lib/python3.12/site-packages/torch/cuda/__init__.py:283: UserWarning: 
    Found GPU0 NVIDIA GB10 which is of cuda capability 12.1.
    Minimum and Maximum cuda capability supported by this version of PyTorch is
    (8.0) - (12.0)
    
  warnings.warn(


🦥 Unsloth Zoo will now patch everything to make training faster!
Unsloth: Could not import trl.trainer.alignprop_trainer: Failed to import trl.trainer.alignprop_trainer because of the following error (look up to see its traceback):
cannot import name 'DDPOStableDiffusionPipeline' from 'trl.models' (/root/src/github.com/elizabetht/language-modeling-from-scratch/.venv/lib/python3.12/site-packages/trl/models/__init__.py)
==((====))==  Unsloth 2026.1.2: Fast Llama patching. Transformers: 4.55.4.
   \\   /|    NVIDIA GB10. Num GPUs = 1. Max memory: 119.699 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu130. CUDA: 12.1. CUDA Toolkit: 13.0. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `llm_int8_enable_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

## Next Steps

Add your FP8 Quantization-Aware Training (QAT) code here. This typically includes:
- Configuring LoRA adapters for parameter-efficient fine-tuning
- Setting up the training dataset
- Configuring the trainer with FP8 quantization settings
- Running the training loop
- Saving and evaluating the quantized model